In [1]:
from src.config import API_KEY, SECRET_KEY
paper = True

In [1]:
import time
print('before')
time.sleep(2)
print('after')

before
after


In [2]:
from datetime import datetime, timedelta
 
import alpaca
from alpaca.data.historical.option import OptionHistoricalDataClient, OptionLatestQuoteRequest
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestQuoteRequest
from alpaca.data.requests import StockLatestQuoteRequest

from alpaca.trading.client import TradingClient, GetAssetsRequest
from alpaca.trading.requests import GetOptionContractsRequest, LimitOrderRequest, MarketOrderRequest, GetOrdersRequest
from alpaca.trading.enums import AssetStatus, ContractType, OrderSide, OrderType, TimeInForce, QueryOrderStatus

In [3]:
trade_client = TradingClient(api_key=API_KEY, secret_key=SECRET_KEY, paper=paper)
stock_data_client = StockHistoricalDataClient(api_key=API_KEY, secret_key=SECRET_KEY)
option_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=SECRET_KEY)

In [4]:
req = StockLatestQuoteRequest(
    symbol_or_symbols='TSLA'
)
stock_data_client.get_stock_latest_quote(req)

{'TSLA': {   'ask_exchange': 'V',
     'ask_price': 319.54,
     'ask_size': 1.0,
     'bid_exchange': 'V',
     'bid_price': 319.07,
     'bid_size': 1.0,
     'conditions': ['R'],
     'symbol': 'TSLA',
     'tape': 'C',
     'timestamp': datetime.datetime(2025, 5, 12, 15, 35, 36, 879858, tzinfo=TzInfo(UTC))}}

In [11]:
from alpaca.data.live import StockDataStream

try:
    live_quote_client = StockDataStream(api_key=API_KEY, secret_key=SECRET_KEY)
except Exception as e:
    print(f"Error initializing LiveDataClient: {e}")
    live_quote_client = None


async def quote_data_handler(data):
    print(data)

# live_quote_client.subscribe_trades(/)
# Output trades to the terminal window
live_quote_client.subscribe_trades(quote_data_handler, 'AAPL') # symbol
live_quote_client.run()


AttributeError: 'NoneType' object has no attribute 'is_running'

In [ ]:
# Create request parameters for the latest option trade
request_params = OptionLatestQuoteRequest(
	symbol="TSLA",  # Replace with the desired symbol
	expiration_date="2023-12-15",  # Replace with the desired expiration date
	strike_price=200.0,  # Replace with the desired strike price
	option_type="call"  # Replace with "call" or "put"
)

# Fetch the latest option trade
latest_trade = option_data_client.get_option_latest_trade(request_params)
latest_trade

In [ ]:
acct = trade_client.get_account()
# print(f"Options approved level: {acct.options_approved_level}")
# print(f"Options trading level: {acct.options_trading_level}")
# print(f"Options Buying power: {acct.options_buying_power}")
acct

In [ ]:
# set max options trading level, risk management
acct_config = trade_client.get_account_configurations()
print(f"max options trading level: {acct_config.max_options_trading_level}")

acct_config.max_options_trading_level = None
trade_client.set_account_configurations(acct_config)

acct_config = trade_client.get_account_configurations()
print(f"max options trading level: {acct_config.max_options_trading_level}")

In [5]:
# active options to buy
req = GetAssetsRequest(
    status = AssetStatus.ACTIVE,
    attributes = "options_enabled"
)

options_enabled_underlyings = trade_client.get_all_assets(req)
print(f"Number of Underlyings with options: {len(options_enabled_underlyings)}")

symbol = 'TSLA'
test = trade_client.get_asset(symbol_or_asset_id=symbol)
test
# print(f"{symbol} has options? {'options_enabled' in test.attributes}")

Number of Underlyings with options: 5831


{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'attributes': ['fractional_eh_enabled', 'has_options'],
    'easy_to_borrow': True,
    'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
    'fractionable': True,
    'id': UUID('8ccae427-5dd0-45b3-b5fe-7ba5e422c766'),
    'maintenance_margin_requirement': 40.0,
    'marginable': True,
    'min_order_size': None,
    'min_trade_increment': None,
    'name': 'Tesla, Inc. Common Stock',
    'price_increment': None,
    'shortable': True,
    'status': <AssetStatus.ACTIVE: 'active'>,
    'symbol': 'TSLA',
    'tradable': True}

In [ ]:
# account positions
positions = trade_client.get_all_positions()
# for p in positions:
#     print(p)
positions

asset_id=UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415') symbol='AAPL' exchange=<AssetExchange.NASDAQ: 'NASDAQ'> asset_class=<AssetClass.US_EQUITY: 'us_equity'> asset_marginable=True avg_entry_price='208.9' qty='1' side=<PositionSide.LONG: 'long'> market_value='208.9156' cost_basis='208.9' unrealized_pl='0.0156' unrealized_plpc='0.0000746768788894' unrealized_intraday_pl='0.0156' unrealized_intraday_plpc='0.0000746768788894' current_price='208.9156' lastday_price='198.53' change_today='0.0523124968518612' swap_rate=None avg_entry_swap_rate=None usd=None qty_available='1'
asset_id=UUID('8ccae427-5dd0-45b3-b5fe-7ba5e422c766') symbol='TSLA' exchange=<AssetExchange.NASDAQ: 'NASDAQ'> asset_class=<AssetClass.US_EQUITY: 'us_equity'> asset_marginable=True avg_entry_price='320.5' qty='1' side=<PositionSide.LONG: 'long'> market_value='320.57' cost_basis='320.5' unrealized_pl='0.07' unrealized_plpc='0.0002184087363495' unrealized_intraday_pl='0.07' unrealized_intraday_plpc='0.0002184087363495' curre

In [ ]:
today = datetime.now().date()
min_expiration = today + timedelta(days=14)
max_expiration = today + timedelta(days=90)

trade_req = StockLatestTradeRequest(symbol_or_symbols=symbol)
latest_trade = stock_data_client.get_stock_latest_trade(trade_req)
last_price = latest_trade[symbol].price

min_strike = str(round(last_price*1.02,2))

print(f"{symbol} last trade price: {last_price}")
print(f"Min exp: {min_expiration}")
print(f"Max exp: {max_expiration}")
print(f"Min strike: {min_strike}")

In [13]:
# create market order
req = MarketOrderRequest(
    symbol='AAPL',
    qty=1,
    side=OrderSide.SELL,
    type=OrderType.MARKET,
    time_in_force=TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

# limit price request, buy when option reaches the limit price
# limit_order = LimitOrderRequest(
#     symbol='SPY',
#     qty=1,
#     side=OrderSide.BUY,
#     time_in_force=TimeInForce.DAY, # will sit until timeInForce determines when to be filled
#     limit_price=486.00
# )
# order = trade_client.submit_order(limit_order)

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'canceled_at': None,
    'client_order_id': '7150887f-0b8d-4e6d-b8da-460311525977',
    'created_at': datetime.datetime(2025, 5, 12, 15, 44, 12, 896312, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 5, 12, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('dbafab75-dfcb-491d-b0c2-2589151bacf9'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.SELL_TO_CLOSE: 'sell_to_close'>,
    'qty': '1',
    'ratio_qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
    'status': <Ord

In [ ]:
# get unfilled orders to manage based off trading algorithm
req = GetOrdersRequest(
    status=QueryOrderStatus.OPEN
)
order = trade_client.get_orders(req)
# print(order)

for elem in order:
    trade_client.cancel_order_by_id(elem.id)

In [ ]:
# close position
symbol = positions[0].symbol
trade_client.close_position(
    symbol_or_asset_id=symbol
)